In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'): 
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/data-eight/Data.xlsx
/kaggle/input/dataseven/Data.xlsx
/kaggle/input/data-six/Data.xlsx
/kaggle/input/datafour/Data.xlsx


In [2]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/data-eight/Data.xlsx
/kaggle/input/dataseven/Data.xlsx
/kaggle/input/data-six/Data.xlsx
/kaggle/input/datafour/Data.xlsx


In [3]:
import pandas as pd
df = pd.read_excel('/kaggle/input/data-eight/Data.xlsx')
df.head()


,Reviews,Category
0,Arena club food buffet has really stepped up a...,Concessions
1,Arena Club staff catering staff very helpful,Concessions
2,Have never used.,Concessions
3,Stm meals have been great this year. Please br...,Concessions
4,We do not buy from the concession.,Concessions


In [4]:
df['Category'].value_counts()

Category
Concessions          270
Video Board/Sound     86
Cheer                 85
Ticketing             79
Fac.                  75
Marketing             73
Parking               68
Name: count, dtype: int64

In [5]:
print(df.columns)

Index(['Reviews', 'Category'], dtype='object')


In [6]:
# Split into features (reviews) and labels (categories)
X = df['Reviews']
Y = df['Category']
len(X)

736

In [7]:
# Split the data into training and test sets
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=22)

In [8]:
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB, ComplementNB
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score, classification_report 

In [9]:

# Define pipelines for different models
pipeMNB = Pipeline([('tfidf', TfidfVectorizer(stop_words='english', ngram_range=(1, 2))), ('clf', MultinomialNB())])
pipeCNB = Pipeline([('tfidf', TfidfVectorizer(stop_words='english', ngram_range=(1,3))), ('clf', ComplementNB())])
pipeSVC = Pipeline([('tfidf', TfidfVectorizer(stop_words='english', ngram_range=(1,3))), ('clf', LinearSVC())])

In [10]:
print(df.isnull().sum())

Reviews     0
Category    0
dtype: int64


In [11]:
df['Reviews'] = df['Reviews'].fillna('')

In [12]:
df['Reviews'] = df['Reviews'].astype(str)

In [13]:
print(X_train.isnull().sum())  # Check for remaining missing values
print(X_train.dtypes)          # Check data types

0
object


In [14]:
pipeMNB.fit(X_train, Y_train)
predictMNB = pipeMNB.predict(X_test)
print(f"MNB Accuracy: {accuracy_score(Y_test, predictMNB):.2f}")

pipeCNB.fit(X_train, Y_train)
predictCNB = pipeCNB.predict(X_test)
print(f"CNB Accuracy: {accuracy_score(Y_test, predictCNB):.2f}")

pipeSVC.fit(X_train, Y_train)
predictSVC = pipeSVC.predict(X_test)
print(f"SVC Accuracy: {accuracy_score(Y_test, predictSVC):.2f}")

MNB Accuracy: 0.45
CNB Accuracy: 0.80
SVC Accuracy: 0.82


In [15]:
from sklearn.preprocessing import LabelEncoder
from xgboost import XGBClassifier
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer

# Encode the target labels
label_encoder = LabelEncoder()
Y_train_encoded = label_encoder.fit_transform(Y_train)
Y_test_encoded = label_encoder.transform(Y_test)

# XGBoost pipeline
pipeXGB = Pipeline([('tfidf', TfidfVectorizer(stop_words = 'english')),
                    ('clf', XGBClassifier(use_label_encoder=False, eval_metric='mlogloss'))])

# Train the model
pipeXGB.fit(X_train, Y_train_encoded)
predictXGB = pipeXGB.predict(X_test)

# Decode the predictions back to original labels
predictXGB_decoded = label_encoder.inverse_transform(predictXGB)

# Print accuracy
print(f"XGB Accuracy: {accuracy_score(Y_test, predictXGB_decoded):.2f}") 

XGB Accuracy: 0.72


In [16]:
msg = "There was a big queue in ticketing office and tickets were expensive"
pipeSVC.predict([msg]) 

array(['Ticketing'], dtype=object)

In [17]:
msg = "I am a student and still paid 28$ to park my car"
pipeSVC.predict([msg]) 

array(['Parking'], dtype=object)

In [18]:
msg = "There should be more giveaways, like horns, t-shirts during volleyball games"
pipeSVC.predict([msg]) 

array(['Marketing'], dtype=object)

In [19]:
import numpy as np
from gensim.models import Word2Vec
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

# Tokenizing reviews
tokenized_train = [doc.split() for doc in X_train]
tokenized_test = [doc.split() for doc in X_test]

# Train Word2Vec model
w2v_model = Word2Vec(sentences=tokenized_train, vector_size=100, window=5, min_count=2, workers=4)

# Function to convert documents to average word vectors
def document_vector(doc):
    # Keep words in the document that are in the Word2Vec vocabulary
    doc = [word for word in doc if word in w2v_model.wv]
    
    # Handle empty documents (no words in vocabulary)
    if len(doc) == 0:
        return np.zeros(w2v_model.vector_size)  # Return a zero vector if no valid words found
    
    # Return the mean of the word vectors
    return np.mean(w2v_model.wv[doc], axis=0)

# Convert training and test documents to vectors
X_train_w2v = np.array([document_vector(doc) for doc in tokenized_train])
X_test_w2v = np.array([document_vector(doc) for doc in tokenized_test])

# Use Logistic Regression since it works with continuous values
model = LogisticRegression(max_iter=1000)
model.fit(X_train_w2v, Y_train)

# Predict and evaluate
predictions = model.predict(X_test_w2v)
print(f"Word2Vec + Logistic Regression Accuracy: {accuracy_score(Y_test, predictions):.2f}")


Word2Vec + Logistic Regression Accuracy: 0.34
